In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
from random import randint
from fake_useragent import UserAgent
import concurrent.futures
from tqdm import notebook
from itertools import repeat
import tqdm



In [ ]:
crawl_data = pd.read_csv('user_review_v1_MOBIUZ.csv',encoding='utf-8')
print(len(crawl_data))

In [ ]:
user_agent = UserAgent()
ua = user_agent

In [ ]:
proxy_list=['45.63.19.165:80','34.134.60.185:443','184.177.84.245:1088','50.250.205.21:32100','67.201.33.10:25283','71.71.162.234:39593','185.59.66.210:33611','181.209.219.28:4153','64.124.191.98:32688',
'12.216.103.114:39072','38.133.200.94:31596','174.108.154.129:32940','70.60.230.8:34116','170.106.155.14:1080','181.129.144.59:3629']
test_proxy = {f'{proxy}':0 for proxy in proxy_list}
print(test_proxy)
def proxy(idx):
    proxies = {"http":"socks4://"+str(proxy_list[idx%len(proxy_list)])}
    return proxies
    

In [ ]:

def get_cookies(asin):
    url = str('https://www.amazon.com/dp/'+str(asin)+'?th=1')
    headers={'User-agent':ua.random}
    res = requests.get(url,headers = headers)
    cookiejar = res.cookies
    return cookiejar

#get_cookies(crawl_data['Asin'][randint(0,50)])
    


In [ ]:
failed_asin=[]

In [ ]:
def request_web(url,idx):
    headers={'User-agent':ua.random,
        'Cookie':str(get_cookies(crawl_data['Asin'][randint(0,50)]))}
    prox = proxy(idx)
    try:
        response = requests.get(url,headers=headers,proxies=prox)
        txt = response.text
        return txt
    except:
        #test_proxy[str(f'{prox}')] += 1
        failed_asin.append(url)
        print('Requests Failed')
        return False
    
        


In [ ]:
def get_content(soap):
    category = soap.find_all("a",class_="a-link-normal a-color-tertiary")
    price = soap.find("span",class_='a-offscreen')
    try:
        brand = soap.find(text='Brand').findNext('td').text
    except:
        brand = 'None'
    if category != []:
        category = [str(t).split('">')[1].split('</a>')[0].replace(" ","").replace('\n',"").replace('&amp;',"") for t in category]
        try:
            price = str(price).split('$')[1].replace('</span>','')
        except:
            price = 'Non-Available'
        content_dict={'category':category,'price':price,'Brand':brand}
        return content_dict
    else:
        #print('Content Failed')
        return False
        
    

In [ ]:

def scrape(arg):
    url,idx = arg
    tmp_format = pd.DataFrame({'Asin':[],'Brand':[],'Category':[],'Sub_category':[],'Breakdown_category':[],'Price':[]})
    if request_web(url,idx) != False:
        soap = BeautifulSoup(request_web(url,idx),"lxml")
        if get_content(soap) != False:
            brand = get_content(soap)['Brand']
            category = get_content(soap)['category']
            price = get_content(soap)['price']
            tmp_format = tmp_format.append({
            'Asin':url.split('/dp/')[1],'Category':[category[1]],'Sub_category':[category[-2]],
            'Breakdown_category':category[-1],'Price':[price],'Brand':[brand]},ignore_index=True)
            time.sleep(3)
        else:
            failed_asin.append(url)
            time.sleep(2)
                
    return tmp_format


In [ ]:
failed_asin=[]
final_format=pd.DataFrame({'Asin':[],'Brand':[],'Category':[],'Sub_category':[],'Breakdown_category':[],'Price':[]})

In [ ]:
def run():
    base_url = 'https://www.amazon.com'
    url_list=[f"{base_url}/dp/{asin}" for asin in crawl_data['Asin']]
    tmp_format = pd.DataFrame({'Asin':[],'Brand':[],'Category':[],'Sub_category':[],'Breakdown_category':[],'Price':[]})
    idx_list=[]
    for idx in range(len(crawl_data)-1):
        idx_list.append(idx)
    arg = list(zip(url_list,idx_list))
    with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
        for result in tqdm.tqdm_notebook((executor.map(scrape,arg)),total=len(url_list)):
           tmp_format = tmp_format.append(result,ignore_index = True)
        executor.shutdown()
    return tmp_format
    


    

In [ ]:
final_format = run()
print('__________________RESULT_____________________')
print('失敗數:'+str(len(failed_asin)),
      'Proxy error:'+str(test_proxy),
      final_format,sep='\n')

In [ ]:
final_format.to_csv('Asin_Category_MOBIUZ_list1.csv')

#### Retry failed URLs in first crawling 
##### Retry Loop until it done

In [ ]:
print(len(failed_asin))

In [ ]:
failed_format = pd.DataFrame({'fail':failed_asin})
failed_format.to_csv('failed_list.csv')

In [ ]:
failed2_asin=pd.read_csv('failed_list.csv')['fail']
failed2_asin

In [ ]:
failed_asin=[]

In [ ]:
def fail_run():
    url_list=[asin for asin in failed2_asin]
    tmp_format = pd.DataFrame({'Asin':[],'Category':[],'Sub_category':[],'Breakdown_category':[],'Price':[]})
    idx_list=[]
    for idx in range(len(failed2_asin)-1):
        idx_list.append(idx)
    arg = list(zip(url_list,idx_list))
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        for result in tqdm.tqdm_notebook((executor.map(scrape,arg)),total=len(url_list)):
           tmp_format = tmp_format.append(result,ignore_index = True)
        executor.shutdown()
    return tmp_format

In [ ]:
retry_format = fail_run()
print('__________________RESULT_____________________')
print('失敗數:'+str(len(failed_asin)),
      'Proxy error:'+str(test_proxy),
      retry_format,sep='\n')


In [ ]:
retry_format

In [ ]:
retry_format.to_csv('Asin_Category_retry.csv')

#### Concat 1st & 2nd Crawling result

In [ ]:
one = pd.read_csv('Asin_Category_MOBIUZ_list1.csv')
retry= pd.read_csv('Asin_Category_retry.csv')


In [ ]:
concat_all = pd.concat([one,retry],ignore_index = True)
print(concat_all)

In [ ]:
concat_all.to_csv('Asin_Category_MOBIUZ_list.csv')